In [1]:
import pandas as pd
import gensim

In [2]:
class Corpus:
    def __init__(self, path):
        self.path = path
    def __iter__(self):
        i = 0
        fp = open(self.path)
        line = fp.readline()
        while line:
            yield line[:-1].split()
            line = fp.readline()
            i+=1
        fp.close()

In [3]:
sentences = Corpus("data/preprocessed.txt")

In [4]:
word2vec = gensim.models.Word2Vec(min_count=10,
                                  sentences=sentences,
                                 size=16,
                                 workers=4)

In [5]:
word2vec.wv.vocab

{'long': <gensim.models.keyedvectors.Vocab at 0x7f997699b518>,
 'time': <gensim.models.keyedvectors.Vocab at 0x7f997699b4a8>,
 '...': <gensim.models.keyedvectors.Vocab at 0x7f997699bc18>,
 'i': <gensim.models.keyedvectors.Vocab at 0x7f997699b470>,
 'have': <gensim.models.keyedvectors.Vocab at 0x7f997699b278>,
 'been': <gensim.models.keyedvectors.Vocab at 0x7f997699bc88>,
 'busy': <gensim.models.keyedvectors.Vocab at 0x7f997699b5c0>,
 'with': <gensim.models.keyedvectors.Vocab at 0x7f9976997668>,
 'work': <gensim.models.keyedvectors.Vocab at 0x7f9976997cf8>,
 ',': <gensim.models.keyedvectors.Vocab at 0x7f9976997b70>,
 'school': <gensim.models.keyedvectors.Vocab at 0x7f9976997978>,
 'rehersal': <gensim.models.keyedvectors.Vocab at 0x7f9976997f98>,
 'other': <gensim.models.keyedvectors.Vocab at 0x7f9976997eb8>,
 'random': <gensim.models.keyedvectors.Vocab at 0x7f99769973c8>,
 'crap': <gensim.models.keyedvectors.Vocab at 0x7f9976997630>,
 'and': <gensim.models.keyedvectors.Vocab at 0x7f9976

In [6]:
import pickle

In [7]:
fp = open("word2vec", "wb")
pickle.dump(word2vec, fp)
fp.close()

In [8]:
fp = open("word2vec", "rb")
word2vec2 = pickle.load(fp)
fp.close()

In [ ]:
word2vec2.v